https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

In [31]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ...,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1,2)
objp

array([[0., 0., 0.],
       [1., 0., 0.],
       [2., 0., 0.],
       [3., 0., 0.],
       [4., 0., 0.],
       [5., 0., 0.],
       [6., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [2., 1., 0.],
       [3., 1., 0.],
       [4., 1., 0.],
       [5., 1., 0.],
       [6., 1., 0.],
       [0., 2., 0.],
       [1., 2., 0.],
       [2., 2., 0.],
       [3., 2., 0.],
       [4., 2., 0.],
       [5., 2., 0.],
       [6., 2., 0.],
       [0., 3., 0.],
       [1., 3., 0.],
       [2., 3., 0.],
       [3., 3., 0.],
       [4., 3., 0.],
       [5., 3., 0.],
       [6., 3., 0.],
       [0., 4., 0.],
       [1., 4., 0.],
       [2., 4., 0.],
       [3., 4., 0.],
       [4., 4., 0.],
       [5., 4., 0.],
       [6., 4., 0.],
       [0., 5., 0.],
       [1., 5., 0.],
       [2., 5., 0.],
       [3., 5., 0.],
       [4., 5., 0.],
       [5., 5., 0.],
       [6., 5., 0.]], dtype=float32)

In [38]:
# Arrays to store object points and image points from all images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

# Load images
images = glob.glob("opencv_images/*.jpg")
images

['opencv_images/left05.jpg',
 'opencv_images/left06.jpg',
 'opencv_images/left04.jpg',
 'opencv_images/left13.jpg',
 'opencv_images/left02.jpg',
 'opencv_images/left01.jpg',
 'opencv_images/left14.jpg',
 'opencv_images/left09.jpg',
 'opencv_images/left03.jpg',
 'opencv_images/left11.jpg',
 'opencv_images/left12.jpg',
 'opencv_images/left08.jpg',
 'opencv_images/left07.jpg']

In [39]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)
    
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(0)

cv2.destroyAllWindows()

In [45]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [47]:
ret, mtx, dist, rvecs, tvecs

(0.15589875911312515,
 array([[534.15663137,   0.        , 341.71479627],
        [  0.        , 534.25492559, 232.05013999],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-2.94269293e-01,  1.23247846e-01,  1.13850492e-03,
         -1.38021877e-04,  1.02084832e-02]]),
 (array([[-0.30060119],
         [ 0.39118933],
         [-1.4350777 ]]),
  array([[ 0.30616402],
         [ 0.50226863],
         [-1.82846255]]),
  array([[ 0.41426793],
         [ 0.65539281],
         [-1.33764399]]),
  array([[-0.43142634],
         [ 0.25379395],
         [-3.08884034]]),
  array([[-0.174203  ],
         [-0.46655317],
         [ 1.34790242]]),
  array([[-0.26431364],
         [-0.39636863],
         [-2.74760817]]),
  array([[-0.24294096],
         [ 0.35446819],
         [ 1.5301419 ]]),
  array([[-0.46015187],
         [-0.08996193],
         [-1.33519357]]),
  array([[-0.32188588],
         [ 0.15850432],
         [-1.2418536 ]])),
 (array([[ 1.59827098],
         [ 3.84026025],

refine the camera matrix based on a free scaling parameter using cv.getOptimalNewCameraMatrix().

In [48]:
img = cv2.imread('opencv_images/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [49]:
newcameramtx, roi

(array([[454.53841463,   0.        , 344.06575118],
        [  0.        , 451.58184808, 232.63616417],
        [  0.        ,   0.        ,   1.        ]]),
 (14, 24, 609, 431))

take an image and undistort it

In [50]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

True